In [9]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

from config import db_password

import time

In [17]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    local_movie = dict(movie)
    alt_titles = {}
    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune–Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in local_movie:
            alt_titles[key] = local_movie[key]
            local_movie.pop(key)
    if len(alt_titles) > 0:
        local_movie['alt_titles'] = alt_titles

#define embedded function to remove a column that is equivalent to another column and pop the value to the remaining column
    def change_column_name (old_key,new_key):
        if old_key in local_movie:
            local_movie[new_key] = local_movie.pop(old_key)
# Dictionary of repeated columns with similar names and what they should be changed to
    old_key_dict = {'Directed by':'Director',
                   'Country of origin':'Country',
                   'Distributed by':'Distributor',
                   'Edited by':'Editor(s)',
                   'Produced by':'Producer',
                    'Producer(s)':'Producer',
                    'Production company(s)':'Productioncompany',
                    'Productioncompanies ':'Productioncompany',
                    'Productioncompany ':'Productioncompany',
                    'Released':'Release date',
                    'Length':'Running time',
                    'Screen story by':'Writer(s)',
                    'Screenplay by':'Writer(s)',
                   'Release Date':'Release date,',
                   'Story by':'Writer(s)',
                   'Theme music composer':'Composer(s)',
                    'Music by':'Composer(s)',
                   'Written by':'Writer(s)'}
    
    for key,value in old_key_dict.items():
       change_column_name(key,value)
                    
    return local_movie


In [24]:
def extract_transform_load(wiki_data, kaggle_data, rating_data):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file)
    ratings = pd.read_csv(rating_data)    

    with open(wiki_data, mode = 'r') as file:
        wiki_movies_raw = json.load(file)
# Filter out the tv shows by removing rows with values in 'No. of episodes', and/or 'No. of seasons','Television series',
# Write list comprehension that filters out TV shows from the dataset
    wiki_movies_raw = [movie for movie in wiki_movies_raw if ('Television series' not in movie) and ('No. of episodes' not in movie) and ('No. of Seasons' not in movie)]
    wiki_movies_cleaner = [clean_movie(movie) for movie in wiki_movies_raw]
    wiki_movies_df = pd.DataFrame(wiki_movies_cleaner)
    
    imdb_id_form = r'(tt\d{7})'
        
    try:            
        wiki_movies_df['imdb_id'] = wiki_movies_df['imbd_link'].str.extract(imdb_data_form)
        wiki_movies_df.drop_duplicates(subset = 'imdb_id',inplace = True)
    except Exception as error:
        print("Exception: {}".format(type(error).__name__))
        print("Exception message: {}".format(error))

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_movies_non_null_columns = [column for column in wiki_movies_df.columns if wiki_movies_df[column].isnull().sum() 
                   < 0.9 * len(wiki_movies_df)]
    wiki_movies_df = wiki_movies_df[wiki_movies_non_null_columns]
    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office_non_null = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office_non_null = box_office_non_null.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'
    # 12. Add the parse_dollars function.
    
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

        # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**6
            return value

        # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):
            s = re.sub('\$|\s|[a-zA-Z]','', s)
            value = float(s) * 10**9
            return value

        # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):
            s = re.sub('\$|,','', s)
            value = float(s)
            return value
        else:
            return np.nan

        # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office'] = box_office_non_null.str.extract(f'({form_one}|{form_two})',
                                                              flags=re.IGNORECASE)[0].apply(parse_dollars)
#     wiki_movies_df.drop('Box office', axis = 1, inplace = True)

        # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget_non_null = wiki_movies_df['Budget'].dropna()
    budget_non_null = budget_non_null.apply(lambda x:  ' '.join(x) if type(x) == list else x)
    budget_non_null = budget_non_null.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    budget_non_null = budget_non_null.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget_non_null.str.extract(f'({form_one}|{form_two})', 
                                                               flags=re.IGNORECASE)[0].apply(parse_dollars)
#     wiki_movies_df.drop('Budget', axis = 1, inplace = True)

        # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date_non_null = wiki_movies_df['Release date'].dropna().apply(lambda x:  ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    release_date_non_null.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)
    wiki_movies_df['release_date'] = pd.to_datetime(release_date_non_null.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})')[0], infer_datetime_format=True)
#     wiki_movies_df.drop('Release date', axis = 1, inplace = True)

        # 16. Clean the running time column in the wiki_movies_df DataFrame.
    running_time_non_null = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    running_time_extract = running_time_non_null.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    running_time_extract = running_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['running_time'] = running_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis = 1, inplace = True)

        # Return three variables. The first is the wiki_movies_df DataFrame



    return wiki_movies_df, kaggle_metadata, ratings


In [25]:
# 17. Create the path to your file directory and variables for the three files.
file_dir = '../../Boot_Camp_files/Module_8'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia-movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [26]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load(wiki_file,kaggle_file,ratings_file)

C:\Users\User\anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Exception: KeyError
Exception message: 'imbd_link'


In [27]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [28]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,year,imdb_link,title,Based on,Starring,Cinematography,Release date,Country,Language,...,Distributor,Editor(s),Producer,Productioncompany,Writer(s),Composer(s),box_office,budget,release_date,running_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,1990.0,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,20th Century Fox,Michael Tronick,"[Steve Perry, Joel Silver]",Silver Pictures,"[David Arnott, James Cappe]","[Cliff Eidelman, Yello]",21400000.0,20000000.0,1990-07-11,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",1990.0,https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,Avenue Pictures,Howard E. Smith,"[Ric Kidney, Robert Redlin]",Avenue Pictures,"[James Foley, Robert Redlin]",Maurice Jarre,2700000.0,6000000.0,1990-05-17,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),1990.0,https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,TriStar Pictures,"[John Bloom, Lois Freeman-Fox]",Daniel Melnick,"[Carolco Pictures, IndieProd Company]","[John Eskow, Richard Rush]",Charles Gross,57718089.0,35000000.0,1990-08-10,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),1990.0,https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,Orion Pictures,Susan E. Morse,Robert Greenhut,NaN,Woody Allen,NaN,7331647.0,12000000.0,1990-12-25,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,1990.0,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",Russell Boyd,"December 19, 1990",US,English,...,Paramount Pictures,David Stiven,John Cornell,NaN,Paul Hogan,Maurice Jarre,6939946.0,25000000.0,1990-12-19,95.0


In [29]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'year',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Cinematography',
 'Release date',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor(s)',
 'Producer',
 'Productioncompany',
 'Writer(s)',
 'Composer(s)',
 'box_office',
 'budget',
 'release_date',
 'running_time']